In [128]:
import pandas as pd
import numpy as np
import time

In [240]:
historico = pd.read_csv('dados/historico2013-17.csv', sep=';')

In [241]:
cc = historico[historico['CURSO'].str.contains("COMPUTAÇÃO")]
cc = cc.filter(items=['PERIODO', 'CODIGO', 'DISCIPLINA', 'MATRICULA'])

print "---> Filtra CC"

---> Filtra CC


In [256]:
cc = cc.loc[ cc['MATRICULA'] < 115000000 ]

print "---> Filtra para períodos anteriores a 2015.1"

---> Filtra para períodos anteriores a 2015.1


In [258]:
global min_mat
min_mat = cc['MATRICULA'].min()

network = {}

In [259]:
PERIODO, CODIGO, DISCIPLINA, MATRICULA = 0, 1, 2, 3

start = time.time()
for grupo, obs in cc.groupby(['PERIODO', 'CODIGO']):
    
    for value_x in obs.get_values():
        #print value_x[PERIODO], value_x[CODIGO], value_x[DISCIPLINA], value_x[MATRICULA]        
        mat_x = value_x[MATRICULA] #- min_mat
        
        for value_y in obs.get_values():
            #print value_y[PERIODO], value_y[CODIGO], value_y[DISCIPLINA], value_y[MATRICULA]
            mat_y = value_y[MATRICULA] #- min_mat
            
            if mat_x != mat_y:
                
                if mat_x not in network:
                    network[mat_x] = {}
                
                if mat_y not in network[mat_x]:
                    network[mat_x][mat_y] = 0
                
                network[mat_x][mat_y] += 1

end = time.time()
print "Demorou {}".format(end - start)

Demorou 7.13259196281


In [260]:
biggest_num_conns = {}

for mat, conns in network.iteritems():
    biggest_num_conns[mat] = int((np.mean(conns.values()) + max(conns.values())) / 2)

In [261]:
same_num_conns = {}

for mat, conns in network.iteritems():
    my_n_conns = biggest_num_conns[mat]
    my_same_n_conns= set(filter(lambda x: conns[x] >= my_n_conns, conns))
    same_num_conns[mat] = my_same_n_conns

In [262]:
def is_on_group(set_mother, me): 
    n_containing = len(list(filter(lambda adj: me in same_num_conns[adj], same_num_conns[set_mother])))
    return (float(n_containing) / len(same_num_conns[set_mother])) >= 0.95    
    
def dfs(m):
    visited[m] = True
    my_components.append(m)
    for adj in same_num_conns[m]:
        if not visited[adj] and is_on_group(m, adj):
            dfs(adj)
            
components = []
visited = {m:False for m in network}
for mat, values in same_num_conns.iteritems():
    if not visited[mat]:
        my_components = []
        dfs(mat)
        components.append(my_components)

In [263]:
clusters = []

for component in components:
    if len(component) > 1:
        for i in range(len(component)-1, -1, -1):
            new_row = {'source': component[i],
                       'target': component[i-1]}
            clusters.append(new_row)

In [264]:
output = pd.DataFrame(columns=('source', 'target'))
output = pd.DataFrame(clusters)
output.to_csv('dados/nets.csv', index=False)